In [1]:
import pandas as pd

In [2]:
data_path = "/Users/gimdong-gu/Desktop/EDAV_project/"

### Station nam with location info

In [3]:
station_loc = pd.read_csv(data_path + "mh_station_coords.csv")

In [4]:
# Clean the data
station_loc["station_name"] = station_loc['station_name'].str.upper()
station_loc["station_name"] = station_loc['station_name'].str.replace("SQUARE","SQ")
station_loc["station_name"] = station_loc['station_name'].str.replace("CENTER","CTR")
station_loc['station_name'] = station_loc['station_name'].str.replace(r"(\d+)(RD)", r"\1")
station_loc['station_name'] = station_loc['station_name'].str.replace(r"(\d+)(TH)", r"\1")

### Station Name without location info

In [5]:
station_name = pd.read_csv(data_path + "Turnstile_2015.csv")

In [6]:
station_name = station_name[["Station", "Line Name", "Division"]]
station_name = station_name.drop_duplicates(subset=["Station", "Line Name", "Division"], keep='first')
station_name = station_name.reset_index(drop=True)

In [7]:
# for i in range(station_name.shape[0]):
#     print((station_name.loc[i,"Station"],station_name.loc[i,"Line Name"],station_name.loc[i,"Division"]))

### Perform automatic match and save it into Dictionary

In [8]:
name_list = list(station_name["Station"])

In [9]:
location_list = list(station_loc["station_name"])

In [10]:
name_dict = {}
location_dict = {}

In [11]:
for i in name_list:
    try:
        name_dict[i] = name_dict[i] + 1
    except KeyError:
        name_dict[i] = 1

In [12]:
for i in location_list:
    try:
        location_dict[i] = location_dict[i] + 1
    except KeyError:
        location_dict[i] = 1

### Get address for stations with unique name

In [13]:
# name_dict
# location_dict
address_book = {}
for i in range(len(location_list)):
    if (location_list[i] in list(location_dict.keys())) and (location_list[i] in list(name_dict.keys())):
        if (location_dict[location_list[i]] == 1) and (name_dict[location_list[i]] == 1):
            address_book[location_list[i]] = (station_loc.loc[i, 'lat'], station_loc.loc[i, 'lon'])
            del location_dict[location_list[i]]
            del name_dict[location_list[i]]

### Match lat-lon info with function "address_up" after comparison

In [14]:
def get_location(df, station_name):
    temp = df[df["station_name"] == station_name]
    temp = temp.reset_index(drop=True)
    return temp

In [15]:
# Use predefined function "get_location"
get_location(station_loc, '103 ST')

,station_name,routes,division,lat,lon
0,103 ST,1,IRT,40.799446,-73.968379
1,103 ST,6,IRT,40.790600,-73.947478
2,103 ST,BC,IND,40.796092,-73.961454


In [16]:
def address_up(name1, name2):
    address_book[name1] = (get_location(station_loc, name2).loc[0, 'lat'], 
                           get_location(station_loc, name2).loc[0, 'lon'])
    del name_dict[name1]
    del location_dict[name2]

In [17]:
address_up('110 ST-CPN','110 ST-CENTRAL PARK NORTH')

In [18]:
address_up('116 ST-COLUMBIA','116 ST-COLUMBIA UNIVERSITY')

In [19]:
address_up('163 ST-AMSTERDM','163 ST - AMSTERDAM AV')

In [20]:
address_up('191 ST','191ST ST')

In [21]:
address_up('34 ST-HUDSON YD','34 ST HUDSON YARDS')

In [22]:
address_up('5 AVE-53 ST','5 AV-53 ST')

In [23]:
address_up('51 ST','51ST ST')

In [24]:
address_up('66 ST-LINCOLN','66 ST-LINCOLN CTR')

In [25]:
address_up('81 ST-MUSEUM','81ST ST - MUSEUM OF NATURAL HISTORY')

In [26]:
address_up("B'WAY-LAFAYETTE",'BROADWAY-LAFAYETTE ST')

In [27]:
address_up('BROOKLYN BRIDGE','BROOKLYN BRIDGE-CITY HALL')

In [28]:
address_up('DYCKMAN-200 ST','DYCKMAN ST-200 ST')

In [29]:
address_up('GRD CNTRL-42 ST','GRAND CENTRAL-42ND ST')

In [30]:
address_up('HARLEM 148 ST','HARLEM-148 ST')

In [31]:
address_up('INWOOD-207 ST','INWOOD - 207 ST')

In [32]:
address_up('MARBLE HILL-225','MARBLE HILL-225 ST')

In [33]:
address_up('W 4 ST-WASH SQ','WEST 4 ST')

In [34]:
address_book

{'110 ST': (40.795020000000001, -73.944249999999997),
 '110 ST-CPN': (40.799075000000002, -73.951821999999979),
 '116 ST-COLUMBIA': (40.807721999999998, -73.964109999999977),
 '157 ST': (40.834040999999999, -73.944890000000001),
 '163 ST-AMSTERDM': (40.836013000000001, -73.939892),
 '175 ST': (40.847391000000002, -73.939704000000006),
 '18 ST': (40.741040000000012, -73.997870999999975),
 '190 ST': (40.859022000000003, -73.934179999999998),
 '191 ST': (40.855224999999997, -73.929411999999999),
 '207 ST': (40.864614000000003, -73.918818999999999),
 '215 ST': (40.869444000000001, -73.915278999999998),
 '3 AV': (40.732849000000002, -73.986121999999995),
 '33 ST': (40.746080999999997, -73.982076000000006),
 '34 ST-HUDSON YD': (40.755882, -74.001909999999995),
 '49 ST': (40.759900999999999, -73.984138999999999),
 '5 AVE-53 ST': (40.760167000000003, -73.975223999999997),
 '51 ST': (40.757106999999998, -73.971919999999997),
 '6 AV': (40.737335000000002, -73.996786),
 '66 ST-LINCOLN': (40.77344

In [35]:
station_name['Location'] = None

In [36]:
for i in address_book.keys():
    idx = station_name[station_name["Station"]==i].index.tolist()[0]
    station_name.loc[idx,'Location'] = address_book[i]

In [37]:
station_name

,Station,Line Name,Division,Location
0,CYPRESS HILLS,J,BMT,None
1,FLATBUSH AVE,25,IRT,None
2,55 ST,D,BMT,None
3,125 ST,ACBD,IND,None
4,CANAL ST,ACE,IND,None
5,WALL ST,23,IRT,None
6,BARCLAYS CENTER,BDNQR2345,IRT,None
7,ST LAWRENCE AVE,6,IRT,None
8,JUNCTION BLVD,7,IRT,None
9,WOODHAVEN BLVD,MR,IND,None


### Start get the lists of stations need to be dropped or kept

In [38]:
manual_station = {}

In [39]:
def save_station(name):
    manual_station[name] = name_dict[name]
    del name_dict[name]

In [40]:
save_station('103 ST')

In [41]:
save_station('116 ST')

In [42]:
save_station('125 ST')

In [43]:
save_station('135 ST')

In [44]:
save_station('137 ST CITY COL')

In [45]:
save_station('137 ST-CITY COL')

In [46]:
save_station('14 ST')

In [47]:
save_station('14 ST-6 AVE')

In [48]:
save_station('14 ST-UNION SQ')

In [49]:
save_station('145 ST')

In [50]:
save_station('155 ST')

In [51]:
save_station('168 ST')

In [52]:
save_station('168 ST-BROADWAY')

In [53]:
save_station('181 ST')

In [54]:
save_station('1 AV')

In [55]:
save_station('1 AVE')

In [56]:
save_station('23 ST')

In [57]:
save_station('28 ST')

In [58]:
save_station('28 ST-BROADWAY')

In [59]:
save_station('2 AV')

In [60]:
save_station('2 AVE')

In [61]:
save_station('34 ST-HERALD SQ')

In [62]:
save_station('34 ST-PENN STA')

In [63]:
save_station('42 ST-BRYANT PK')

In [64]:
save_station('42 ST-GRD CNTRL')

In [65]:
save_station('42 ST-PA BUS TE')

In [66]:
save_station('42 ST-PORT AUTH')

In [67]:
save_station('42 ST-TIMES SQ')

In [68]:
save_station('47-50 ST-ROCK')

In [69]:
save_station('47-50 STS ROCK')

In [70]:
save_station('5 AV/53 ST')

In [71]:
save_station('5 AV/59 ST')

In [72]:
save_station('5 AVE-59 ST')

In [73]:
save_station('5 AVE-BRYANT PK')

In [74]:
save_station('5 AVE')

In [75]:
save_station('50 ST')

In [76]:
save_station('57 ST')

In [77]:
save_station('57 ST-7 AV')

In [78]:
save_station('57 ST-7 AVE')

In [79]:
save_station('59 ST')

In [80]:
save_station('59 ST COLUMBUS')

In [81]:
save_station('59 ST-COLUMBUS')

In [82]:
save_station('68ST-HUNTER CO')

In [83]:
save_station('68ST-HUNTER COL')

In [84]:
save_station('7 AVE')

In [85]:
save_station('7 AVE-53 ST')

In [86]:
save_station('7 AV-PARK SLOPE')

In [87]:
save_station('7 AV')

In [88]:
save_station('72 ST')

In [89]:
save_station('77 ST')

In [90]:
save_station('79 ST')

In [91]:
save_station('8 AV')

In [92]:
save_station('8 AVE')

In [93]:
save_station("8 ST-B'WAY NYU")

In [94]:
save_station('8 ST-NYU')

In [95]:
save_station('86 ST')

In [96]:
save_station('96 ST')

In [97]:
save_station('CANAL ST')

In [98]:
save_station('CATHEDRL-110 ST')

In [99]:
save_station('CATHEDRAL PKWY')

In [100]:
save_station('CHAMBERS ST')

In [101]:
save_station('CHRISTOPHER ST')

In [102]:
save_station('DYCKMAN ST')

In [103]:
save_station('FULTON ST')

In [104]:
save_station('GRAND ST')

In [105]:
save_station('LEXINGTON AV/53')

In [106]:
save_station('LEXINGTON AV/63')

In [107]:
save_station('LEXINGTON AVE')

In [108]:
save_station('LEXINGTON-53 ST')

In [109]:
save_station('PARK PLACE')

In [110]:
save_station('RECTOR ST')

In [111]:
save_station('ROOSEVELT IS')

In [112]:
save_station('ROOSEVELT ISLND')

In [113]:
save_station('SPRING ST')

In [114]:
save_station('TIMES SQ-42 ST')

In [115]:
save_station('WALL ST')

In [116]:
# list of stations not in Manhattan
drop_list = list(name_dict.keys())

### Drop rows with stations that are not in Manhattan

In [117]:
station_name = station_name.loc[~station_name["Station"].isin(drop_list)]
station_name = station_name.reset_index(drop=True)

In [118]:
station_name

,Station,Line Name,Division,Location
0,125 ST,ACBD,IND,None
1,CANAL ST,ACE,IND,None
2,WALL ST,23,IRT,None
3,CHAMBERS ST,ACE23,IND,None
4,28 ST-BROADWAY,NR,BMT,None
5,8 ST-B'WAY NYU,NR,BMT,None
6,42 ST-BRYANT PK,BDFM7,IND,None
7,81 ST-MUSEUM,BC,IND,"(40.781433, -73.972143)"
8,34 ST-PENN STA,ACE,IND,None
9,42 ST-TIMES SQ,1237ACENQRS,IRT,None


In [119]:
cnt = 0
for i in range(station_name.shape[0]):
    if station_name.loc[i,"Location"] == None:
        cnt += 1
cnt

142

In [122]:
address_book

{'110 ST': (40.795020000000001, -73.944249999999997),
 '110 ST-CPN': (40.799075000000002, -73.951821999999979),
 '116 ST-COLUMBIA': (40.807721999999998, -73.964109999999977),
 '157 ST': (40.834040999999999, -73.944890000000001),
 '163 ST-AMSTERDM': (40.836013000000001, -73.939892),
 '175 ST': (40.847391000000002, -73.939704000000006),
 '18 ST': (40.741040000000012, -73.997870999999975),
 '190 ST': (40.859022000000003, -73.934179999999998),
 '191 ST': (40.855224999999997, -73.929411999999999),
 '207 ST': (40.864614000000003, -73.918818999999999),
 '215 ST': (40.869444000000001, -73.915278999999998),
 '3 AV': (40.732849000000002, -73.986121999999995),
 '33 ST': (40.746080999999997, -73.982076000000006),
 '34 ST-HUDSON YD': (40.755882, -74.001909999999995),
 '49 ST': (40.759900999999999, -73.984138999999999),
 '5 AVE-53 ST': (40.760167000000003, -73.975223999999997),
 '51 ST': (40.757106999999998, -73.971919999999997),
 '6 AV': (40.737335000000002, -73.996786),
 '66 ST-LINCOLN': (40.77344

In [120]:
manual_station

{'1 AV': 1,
 '1 AVE': 1,
 '103 ST': 3,
 '116 ST': 3,
 '125 ST': 4,
 '135 ST': 2,
 '137 ST CITY COL': 1,
 '137 ST-CITY COL': 1,
 '14 ST': 3,
 '14 ST-6 AVE': 1,
 '14 ST-UNION SQ': 2,
 '145 ST': 3,
 '155 ST': 2,
 '168 ST': 2,
 '168 ST-BROADWAY': 2,
 '181 ST': 2,
 '2 AV': 1,
 '2 AVE': 1,
 '23 ST': 5,
 '28 ST': 3,
 '28 ST-BROADWAY': 1,
 '34 ST-HERALD SQ': 2,
 '34 ST-PENN STA': 3,
 '42 ST-BRYANT PK': 1,
 '42 ST-GRD CNTRL': 1,
 '42 ST-PA BUS TE': 2,
 '42 ST-PORT AUTH': 2,
 '42 ST-TIMES SQ': 2,
 '47-50 ST-ROCK': 1,
 '47-50 STS ROCK': 1,
 '5 AV/53 ST': 1,
 '5 AV/59 ST': 1,
 '5 AVE': 1,
 '5 AVE-59 ST': 1,
 '5 AVE-BRYANT PK': 1,
 '50 ST': 3,
 '57 ST': 1,
 '57 ST-7 AV': 1,
 '57 ST-7 AVE': 1,
 '59 ST': 3,
 '59 ST COLUMBUS': 2,
 '59 ST-COLUMBUS': 2,
 '68ST-HUNTER CO': 1,
 '68ST-HUNTER COL': 1,
 '7 AV': 3,
 '7 AV-PARK SLOPE': 1,
 '7 AVE': 1,
 '7 AVE-53 ST': 1,
 '72 ST': 2,
 '77 ST': 2,
 '79 ST': 2,
 '8 AV': 2,
 '8 AVE': 2,
 "8 ST-B'WAY NYU": 1,
 '8 ST-NYU': 1,
 '86 ST': 5,
 '96 ST': 3,
 'CANAL ST': 3

In [121]:
station_name

,Station,Line Name,Division,Location
0,125 ST,ACBD,IND,None
1,CANAL ST,ACE,IND,None
2,WALL ST,23,IRT,None
3,CHAMBERS ST,ACE23,IND,None
4,28 ST-BROADWAY,NR,BMT,None
5,8 ST-B'WAY NYU,NR,BMT,None
6,42 ST-BRYANT PK,BDFM7,IND,None
7,81 ST-MUSEUM,BC,IND,"(40.781433, -73.972143)"
8,34 ST-PENN STA,ACE,IND,None
9,42 ST-TIMES SQ,1237ACENQRS,IRT,None


In [124]:
station_name.to_csv("subway_clean.csv", index=False)